### FRB Search Volume

Given a DM limit of an FRB search, compute the search volume of the ALFABURST survey. This is a simplified version to the FRB search volume notebook which uses the NE2001 model. We assume here that we are sensitivity rather than maximum DM limited.

Adapted from code by Jayanth Chennamangalam and Evan Keane (calcfrbvol.py)

**Assumptions:**
1. All FRBs originate outside the Galaxy
2. There exists a constant electron density in the IGM (Ioka 2003 and Inoue 2004)
3. Distances are cosmological, $d(z) = \frac{c \cdot z}{H_0}$
4. The number density of galaxies is $n_{\textrm{gal}} = \frac{0.01}{\textrm{Mpc}^{3}}$
5. The host galaxy provides a DM of 100
6. Using a constant z to DM relation of $z = \frac{\textrm{DM}}{1200}$
7. FRBs are flat spectrum, standard candles
8. The maximum DM search is sufficiently large that the galactic DM contribution is small enough that our search volume is sensitivity limited rather than DM limited. True for searches where the maximum DM is 4000+.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units
from astropy.coordinates import SkyCoord
import scipy.integrate

%matplotlib inline

In [12]:
nGalaxies = 1e-2 # number density of galaxies, Mpc^-3

# beam size from ALFA_Beam_Response.ipynb
beamArea =  np.array([0.00687408,  0.00953674,  0.01215363,  0.01485443,  0.0176506 ,  0.02048111,
                      0.02347946,  0.02649689,  0.02993774,  0.0336113 ,  0.03687668,  0.04005814,
                      0.04347992,  0.0476532 ,  0.05156708,  0.05595016,  0.05998993,  0.06488037,
                      0.07263184,  0.08158875,  0.09100342,  0.10020065,  0.10749054,  0.11266708,
                      0.11685944,  0.12044144,  0.12305069,  0.12533569,  0.12727356,  0.12876129,
                      0.13031387,  0.13171005,  0.13315582,  0.13444138,  0.13581848,  0.13701248,
                      0.13830948,  0.139534  ,  0.14064789,  0.14182281,  0.14297485,  0.14406204,
                      0.14522934,  0.1462822 ,  0.1473999 ,  0.14834213,  0.14943695,  0.15048218,
                      0.15138245,  0.15245819,  0.15339279,  0.15434647,  0.1553688 ,  0.15626907,
                      0.15717316,  0.15817642,  0.15905762,  0.15990067,  0.16085052,  0.16173172,
                      0.16257858,  0.16348267,  0.16431808,  0.16512299])
beamArea *= ((np.pi/180.)**2.)

for ba in beamArea:
    print 'Beam Area: %f deg^2 (%f str)'%(ba * ((180./np.pi)**2.), ba)

Beam Area: 0.006874 deg^2 (0.000002 str)
Beam Area: 0.009537 deg^2 (0.000003 str)
Beam Area: 0.012154 deg^2 (0.000004 str)
Beam Area: 0.014854 deg^2 (0.000005 str)
Beam Area: 0.017651 deg^2 (0.000005 str)
Beam Area: 0.020481 deg^2 (0.000006 str)
Beam Area: 0.023479 deg^2 (0.000007 str)
Beam Area: 0.026497 deg^2 (0.000008 str)
Beam Area: 0.029938 deg^2 (0.000009 str)
Beam Area: 0.033611 deg^2 (0.000010 str)
Beam Area: 0.036877 deg^2 (0.000011 str)
Beam Area: 0.040058 deg^2 (0.000012 str)
Beam Area: 0.043480 deg^2 (0.000013 str)
Beam Area: 0.047653 deg^2 (0.000015 str)
Beam Area: 0.051567 deg^2 (0.000016 str)
Beam Area: 0.055950 deg^2 (0.000017 str)
Beam Area: 0.059990 deg^2 (0.000018 str)
Beam Area: 0.064880 deg^2 (0.000020 str)
Beam Area: 0.072632 deg^2 (0.000022 str)
Beam Area: 0.081589 deg^2 (0.000025 str)
Beam Area: 0.091003 deg^2 (0.000028 str)
Beam Area: 0.100201 deg^2 (0.000031 str)
Beam Area: 0.107491 deg^2 (0.000033 str)
Beam Area: 0.112667 deg^2 (0.000034 str)
Beam Area: 0.116

In [13]:
# Constants
deg2rad = np.pi / 180.
rad2deg = 180. / np.pi
Gpc32Mpc3 = 1e9 # convert Gpc^3 to Mpc^3

In [14]:
def coMovingDist(z):
    """Co-moving distance in Gpc, Eq. 3 from Lorimer et al. 2013
    z: float, redshift
    """
    c = 299792.458 # km s^-1
    H0 = 68. # km s^-1 Mpc^-1
    OmegaM = 0.32
    OmegaLambda = 0.68
    
    integrand = lambda zp: 1. / np.sqrt(OmegaM * ((1. + zp)**3.) + OmegaLambda)
    dd, err = scipy.integrate.quad(integrand, 0., z)
    return ((c/H0) * dd) * 1e-3

In [19]:
#for l,b in zip(gall[:10],galb[:10]):
print 'Pointings:', gall.shape[0]
for cnt,(l,b) in enumerate(zip(gall,galb)):
    dd0 = coMovingDist(3.2)
    vol0 = (np.pi * ((dd0 * (beamFWHM0/2.))**2.) * dd0) / 3.
    
    dmDiff = dmMax - (dmGal + dmHost) # > 0 if there is excess DM outside the host and our galaxy
    if dmDiff > 0.:
        #zz = dmDiff / 1200. # calculate the distance in z to the source using the relation DM ~ 1200 z cm^-3 pc
        # for z <= 2 (Lorimer et al. 2007), based on Ioka (2003) and Inoue (2004)
        zz = 1.5
        
        zVals.append(zz)
        
        dd = coMovingDist(zz) # compute the Hubble distance, in Gpc
        
        # Volume, assuming a cone, V = pi r^2 h / 3, where r = dd theta / 2 (small angle approximation), and h = dd
        # this ignores any sensitivity beyond the beam FWHM
        vol = (np.pi * ((dd * (beamFWHM/2.))**2.) * dd) / 3. - (np.pi * ((dd * (beamFWHM0/2.))**2.) * dd) / 3. +vol0
        vol *= Gpc32Mpc3 # convert from Gpc^3 to Mpc^3
        nGal = vol * nGalaxies  # number of galaxies in volume
        
        nGalSurvey += nGal
        volSurvey += vol
        areaSurvey += areaDeg

zVals = np.array(zVals)
zMin = np.min(zVals)
zMax = np.max(zVals)
zMean = np.mean(zVals)

Pointings: 63844


NameError: name 'beamFWHM0' is not defined

In [9]:
print "Total time observing = %i s"%(totalObsTime)
print "Observing time outside the galactic plane (+/-%.1f degrees) = %i s"%(gal_cut, galCutObsTime)
print "Total volume surveyed = %2.3e Gpc^3" % (volSurvey / Gpc32Mpc3), "= %2.3e Mpc^3" % volSurvey
print "Total area surveyed = %2.3e sq. deg." % areaSurvey
print "z (max = %2.3f, min = %2.3f, mean = %2.3f)" % (zMax, zMin, zMean)
print "Total number of galaxies (assuming n = %2.3f" % nGalaxies, "Mpc^-3) = %2.3f" % nGalSurvey
print dd
print vol
print nGal

Total time observing = 3830640 s
Observing time outside the galactic plane (+/-7.0 degrees) = 2175300 s
Total volume surveyed = 2.199e+02 Gpc^3 = 2.199e+11 Mpc^3
Total area surveyed = 4.250e+00 sq. deg.
z (max = 1.500, min = 1.500, mean = 1.500)
Total number of galaxies (assuming n = 0.010 Mpc^-3) = 2198741088.309
4.42646102365
6064656.15311
60646.5615311


In [10]:
500000. / (60.*60.*24*365)

0.015854895991882292

In [11]:
print nGal

60646.5615311


In [12]:
print nGal * 1e-4 * 0.12146879756468798

0.736666490561
